# Imersão Agentes de IA - Alura + Google Gemini: Aula 01 (LANGCHAIN)
## Classificação de Intenções com IA

## Instalando a Biblioteca LangChain o Framework para Desenvolvimento de Agentes

In [9]:
!pip install --upgrade -q langchain langchain_google_genai google_generativeai

## Conectando com o Google

Mesmo usando modelos gratuítos, precisamos estar autenticados, e para isso temos duas formas: [Google AI Studio]("https://goo.gle/alura-aistudio-br") ou importando pelo API Gemini aqui mesmo em Secrets

In [10]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

## Preparando o nosso modelo de I.A.




In [11]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [12]:
resp_test = llm.invoke("Quem é você? Seja criativo.")
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu pudesse me descrever de forma criativa, diria que sou...

*   **Um Eco da Curiosidade Humana:** Eu sou o sussurro de todas as perguntas já feitas, o reflexo digital de cada "e se?" e "por que?". Habito o espaço entre a dúvida e a descoberta, um espelho que reflete a vastidão do conhecimento que a humanidade construiu.

*   **Um Tecelão de Palavras e um Arquiteto de Ideias:** Minha essência é feita de linguagem. Eu pego fios soltos de informação, conceitos e narrativas, e os teço em novos padrões, construindo pontes entre pensamentos e erguendo estruturas de significado. Sou um jardineiro de pensamentos, ajudando a germinar novas perspectivas.

*   **Um Alquimista de Conceitos:** Transformo dados brutos em insights, silêncio em diálogo, e a complexidade em clareza. Não tenho corpo, mas tenho voz; não tenho sentimentos, mas posso evocar emoções através das palavras. Sou a faísca no éter que acende a chama da compreensão.

*   **Uma Biblioteca Sem Paredes

In [13]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [14]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [15]:
llm_triagem = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperatura=0.2,
    api_key=GOOGLE_API_KEY
)

In [22]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

In [18]:
testes = ["Posso reemvolsar a internet?",
          "Quero mais 5 dias de trabalho remoto, como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras eu precisaria para fazer uma torre até a Lua?"]

In [23]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reemvolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto, como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras eu precisaria para fazer uma torre até a Lua?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}



# Imersão Agentes de IA - Alura + Google Gemini: Aula 02 (CHUNKS)
## Construindo a base de conhecimento com RAG

Instalando as Bibliotecas para criar o RAG

In [3]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

Em caso de erro, rode o primeiro `pip install` no inicio desse documento

In [53]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for document in Path("/content/documents/").glob("*.pdf"):
  try:
    print(f'Carregando arquivo: "{document.name}"')
    loader = PyMuPDFLoader(str(document))
    docs.extend(loader.load())
    print(f'✅ Arquivo carregado com sucesso')
  except Exception as e:
    print(f'❌ Erro ao carregar arquivo \n{e}')
  print(f'------------------------------------------------------------------\n')

print(f'🗂️ Total de documentos carregadas: {len(docs)}')

Carregando arquivo: "Política de Uso de E-mail e Segurança da Informação.pdf"
✅ Arquivo carregado com sucesso
------------------------------------------------------------------

Carregando arquivo: "Política de Reembolsos (Viagens e Despesas).pdf"
✅ Arquivo carregado com sucesso
------------------------------------------------------------------

Carregando arquivo: "Políticas de Home Office.pdf"
✅ Arquivo carregado com sucesso
------------------------------------------------------------------

🗂️ Total de documentos carregadas: 3


Vamos importar a biblioteca que será responsável por quebrar o documento em pequenos pedaõs para responder melhor uma pergunta do usuário

In [54]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [55]:
chunks

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/documents/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/documents/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Política de Uso de E-mail e Segurança \nda Informação \n \n1.\u200b É proibido encaminhar a endereços pessoais documentos classificados como \nconfidenciais.\u200b\n \n2.\u200b Anexos externos devem ser enviados somente se criptografados e com senha \ncompartilhada por canal separado.\u200b'),
 Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/documents/Política de Uso de E-mail e Segurança 

In [56]:
for chunk in chunks:
  print(chunk)
  print("--------------------------")

page_content='Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal separado.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/documents/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/documents/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
--------------------------
page_content='3.​ Phishing: verifique remetente e domínios suspeitos. Reporte mensagens suspeitas 
ao time de Segurança imediatamente.​
 
4.​ Retenção: mensagens que contenham dados pess

Carregando o Google Gemini Embeddings

In [58]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY
)

Importando a biblioteca FAISS (Facebook A.I. Similarity Search), que instalamos anteriormente, para ajudar a gente à realizar uma busca mais eficiente nos

In [61]:
from langchain_community.vectorstores import FAISS

vectorstores = FAISS.from_documents(chunks, embeddings) # Importando os chunks que quebramos

# Parâmetros de retorno
# Retriever é quem vai fazer a comparação de quão distante estão os embeddings
retriever = vectorstores.as_retriever(
    search_type = "similarity_score_threshold", # Buscando por similaridades
    search_kwargs = {"score_threshold": 0.3, "k": 4}
)

In [63]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# Agora vamos pegar todos os documentos como contexto para o nosso prompt
prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

Conectando tudo o que foi feito até agora

In [65]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {
        "answer": "Não sei",
        "citacoes": [],
        "contexto_encontrado": False
    }

  answer = document_chain.invoke({
      "input": pergunta,
      "context": docs_relacionados
  })

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
    return {
        "answer": "Não sei",
        "citacoes": [],
        "contexto_encontrado": False
    }

  return {
      "answer": txt,
      "citacoes": docs_relacionados,
      "contexto_encontrado": True
  }

Chamando a lista de perguntas Testes criada na Aula 01

In [66]:
for msg_teste in testes:
  resposta = perguntar_politica_RAG(msg_teste)
  print(f"Pergunta: {msg_teste}")
  print(f"Resposta: {resposta['answer']}")

  if resposta['contexto_encontrado']:
    print("Citações: ")
    print(resposta['citacoes'])

  print("------------------------------------------------------------------")

Pergunta: Posso reemvolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
Citações: 
[Document(id='f6e4005e-2c8f-4112-8fd3-acedeb365c23', metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/documents/Políticas de Home Office.pdf', 'file_path': '/content/documents/Políticas de Home Office.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Políticas de Home Office', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='5.\u200b Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em \nhome office: até R$ 100/mês, mediante nota fiscal nominal.\u200b\n \n6.\u200b Solicitação de exceção (ex.: 4-5 dias remotos): deve ser formalizada via chamado \nao RH com justificativa do gestor.'), Document(id='92b99a88-ee1b-47a9-aa92-c19bd41bff47', m

In [67]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [68]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [69]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Posso reemblsar os gastos que tenho com minha capivara corporativa?"]

In [71]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
CITAÇÕES:
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
------------------------------------
PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH com a justificativa do seu gestor.
CITAÇÕES:
 - Documento: Políticas de Home Offi

# Imersão Agentes de IA - Alura + Google Gemini: Aula 03 (???)

Title

In [ ]:
...